In [255]:
import pandas as pd
import time
import requests
import pickle
from bs4 import BeautifulSoup
from cookies import *

In [22]:
# GET COOCKIES AND HEADER FROM BROWSER AFTER U LOGIN
response = requests.get('https://etherscan.io/labelcloud', headers=headers, cookies=cookies)
soup = BeautifulSoup(response.text, 'html.parser')
sources = []
for a in soup.find_all("a"):
    try:
        for href in a.get_attribute_list('href'):
            try:
                if '/label/' in href:
                    amt = int(a.text.split(' ')[-1].replace("(", "").replace(")", ""))
                    sources.append([href, amt])
            except:
                continue
    except:
        continue

In [63]:
#results = {}
for pair in sources:
    link = pair[0]
    amt = pair[1]
    full_url = "https://etherscan.io"+link
    params = (
        ('subcatid', 'undefined'),
        ('size', str(amt)),
        ('start', '0'),
        ('col', '1'),
        ('order', 'asc'),
    )
    if 'accounts' in full_url:
        response = requests.get(full_url, headers=headers, cookies=cookies, params=params)
        if response.status_code == 200:
            #print("Complete %s" % full_url)
            results[full_url] = response.text
        else:
            print("Error %s - %i" % (full_url, response.status_code))
    else:
        response = requests.get(full_url+'?subcatid=undefined&size=100&start=0&col=3&order=desc', headers=headers, cookies=cookies)
        if response.status_code == 200:
            if 'amounts of traffic coming from your network' in response.text:
                print("Error %s"  % value)
            else:
                #print(full_url)
                results[full_url] = response.text
        else:
            print("Error %s"  % value)
    time.sleep(2)

Complete https://etherscan.io/accounts/label/0-gas-sender
Complete https://etherscan.io/accounts/label/0x
Complete https://etherscan.io/tokens/label/0x
Complete https://etherscan.io/accounts/label/0x-ecosystem
Complete https://etherscan.io/tokens/label/0x-ecosystem
Complete https://etherscan.io/accounts/label/0xuniverse
Complete https://etherscan.io/tokens/label/0xuniverse
Complete https://etherscan.io/accounts/label/1inch-exchange
Complete https://etherscan.io/tokens/label/1inch-exchange
Complete https://etherscan.io/tokens/label/3d-printing
Complete https://etherscan.io/accounts/label/aave
Complete https://etherscan.io/tokens/label/aave
Complete https://etherscan.io/accounts/label/aave-v1
Complete https://etherscan.io/tokens/label/aave-v1
Complete https://etherscan.io/accounts/label/aave-v2
Complete https://etherscan.io/tokens/label/aave-v2
Complete https://etherscan.io/accounts/label/abcc
Complete https://etherscan.io/tokens/label/abcc
Complete https://etherscan.io/tokens/label/adul

In [142]:
# import pickle
# pickle.dump(results, open('results.p', 'wb'))

In [250]:
df_as_list = []
for url,text in results.items():
    if '/txs/' in url or '/blocks/' in url:
        continue
    soup = BeautifulSoup(text, 'html.parser')
    # family
    family = url.split('/')[-1]
    # type
    typex = ''
    website = ''
    if '/accounts/' in url:
        typex = 'account'
    elif '/tokens/' in url:
        typex = 'tokens'
    # fields
    rows = soup.select("tbody tr")
    if len(rows)==0:
        print(url)
    for row in rows:
        fields = list(row.children)
        token_market_cap = ''
        token_num_holders = ''
        tx_count =  ''
        balance  =  ''
        website = ''
        if typex == 'tokens':
            token_market_cap = fields[3].text
            token_num_holders = fields[4].text
            name = fields[2].text
            address  =  fields[1].text
            website = list(fields[5].children)[0]['href'] if fields[5].text != '-'  else ''
        elif typex == 'account':
            if fields[2].text == 'Wrapped Ether':
                break
            tx_count = int(fields[3].text.split(" ")[0].replace(",",""))
            balance=  fields[2].text.split(" ")[0]
            name = fields[1].text
            address  = fields[0].text
        df_as_list.append({
            'address': address, # address
            'name': name, # name if any 
            'family': url.split('/')[-1], # category
            'type': typex, # token or address
            'website': website, # url if token
            'token_market_cap': token_market_cap, # if a token then what is the market cap
            'token_num_holders': token_num_holders, # if its a token then how many holders
            'tx_count': tx_count, # if an address (not a token) then how many txs has it done
            'balance': balance, # if an address (not a token) then current ETH balance
        })

https://etherscan.io/accounts/label/education
https://etherscan.io/accounts/label/electronics
https://etherscan.io/accounts/label/media
https://etherscan.io/accounts/label/music
https://etherscan.io/accounts/label/prediction-market
https://etherscan.io/tokens/label/remittance+
https://etherscan.io/tokens/label/set-social-trading
https://etherscan.io/tokens/label/token-contract


In [253]:
df = pd.DataFrame(df_as_list)
df.to_csv("etherscan_tags.csv", index=False)

In [268]:
exchange_defi = df[(df['family']=='exchange') | (df['family']=='defi')]['address'].values

In [269]:
exchange_defi

array(['...'], dtype=object)

In [307]:
users = pd.read_csv("../bigquery/forsage_users.csv", names=['address'])
users = set(users['address'].values)
len(users)

1064144

In [308]:
exchanges = set([x.replace(" ","") for x in exchange_defi])
len(exchanges)

407

In [313]:
len(users-exchanges)

1064133

In [314]:
len(users-set(df['address']))

1064114

In [320]:
df[df['family']=='binance']

,address,name,family,type,website,token_market_cap,token_num_holders,tx_count,balance
4003,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,Binance,binance,account,,,,15840129,"188,172.71830951"
4004,0x85b931a32a0725be14285b66f1a22178c672d69b,Binance 10,binance,account,,,,742795,"22,100.1515808"
4005,0x708396f17127c42383e3b9014072679b2f60b82f,Binance 11,binance,account,,,,752945,"8,066.25800872"
4006,0xe0f0cfde7ee664943906f17f7f14342e76a5cec7,Binance 12,binance,account,,,,193678,"19,243.09405396"
4007,0x8f22f2063d253846b53609231ed80fa571bc0c8f,Binance 13,binance,account,,,,192397,"13,164.53309176"
4008,0xd551234ae421e3bcba99a0da6d736074f22192ff,Binance 2,binance,account,,,,4495290,"20,937.68051839"
4009,0x564286362092d8e7936f0549571a803b203aaced,Binance 3,binance,account,,,,4381947,"8,256.91980244"
4010,0x0681d8db095565fe8a346fa0277bffde9c0edbbf,Binance 4,binance,account,,,,4476073,"21,396.3326052"
4011,0xfe9e8709d3215310075d67e3ed32a380ccf451c8,Binance 5,binance,account,,,,13401,0.06523058
4012,0x4e9ce36e442e55ecd9025b9a6e0d88485d628a67,Binance 6,binance,account,,,,395,0.10383631


In [319]:
results['https://etherscan.io/accounts/label/binance']

'\n<!doctype html>\n<html lang="en">\n<head><title>\r\n\tEthereum Top Accounts by ETH Balance | Etherscan\r\n</title><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" /><meta name="Description" content="The top accounts in Ethereum in descending order by the number of Ether (ETH) balance in an account and the percentage of the Ether (ETH) that the account hold." /><meta name="author" content="etherscan.io" /><meta name="keywords" content="ethereum, explorer, ether, search, blockchain, crypto, currency" /><meta name="format-detection" content="telephone=no" />\n<meta property="og:title" content="Ethereum Top Accounts by ETH Balance | Etherscan" /> <meta property="og:description" content="The top accounts in Ethereum in descending order by the number of Ether (ETH) balance in an account and the percentage of the Ether (ETH) that the account hold." /> <meta property="og:type" content="website" /> <meta property="og:site_name" con